# TP1
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Problema 1

### Descrição do Problema

Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
    1. Cada reunião ocupa uma sala (enumeradas $$1...S\,$$) durante um “slot”  $$1..T\,$$ $$(\text{hora},\text{dia})$$.  
    2.  Cada reunião tem associado um projeto (enumerados $$1..P$$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $$1..C$$.
    3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. 
    4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  $$50\%$$ do total de colaboradores do projeto. 
    
    
    ( falta deixar isto mais clean)
    

### Código Python

In [1]:
from ortools.linear_solver import pywraplp
horario = pywraplp.Solver.CreateSolver('SCIP')

Criação da matriz de alocação:

Construção da matriz 2x2 da disponibilidade de cada participante:

In [2]:

import random
P,C,S,T = 3,8,3,6

def printmatrix(x,m,n):
    for i in range(m):
        for j in range(n):
            print(x[i,j], end = ' ')  
        print()

def creatematrix(x,string,m,n):
    l = 0
    for i in range(m):
        for j in range(n):
            x[i,j] = int(string[l])
            l += 1

def createdict(x,string,n):
    for i in range(n):
        x[i] = int(string[i])

d = {}
s = '110110110110110011011011011101111011110111001100'
creatematrix(d,s,C,T)
#for i in range(P):
#    for j in range(C):
#        d[(i,j)] = random.randrange(1,2)
    

l = {}
s = '036'
createdict(l,s,P)
#n = random.sample(range(5),5)
#for i in range(5):
#    l[i] = n[i]
    
col = {}
#for i in range(5):
#    for j in range(5):
#        col[(i,j)] = random.randrange(0,2)
s = '111000000011111000000111'
creatematrix(col,s,P,C)

s = '311'
m = {}
createdict(m,s,P)
#for i in range(P):
#    m[i] = random.randrange(1,2)

    
print('D')
printmatrix(d,C,T)
print()
print('L')    
print(l)
print()
print('COL')    
printmatrix(col,P,C)
print()
print('M')    
print(m)
print()

D
1 1 0 1 1 0 
1 1 0 1 1 0 
1 1 0 0 1 1 
0 1 1 0 1 1 
0 1 1 1 0 1 
1 1 1 0 1 1 
1 1 0 1 1 1 
0 0 1 1 0 0 

L
{0: 0, 1: 3, 2: 6}

COL
1 1 1 0 0 0 0 0 
0 0 1 1 1 1 1 0 
0 0 0 0 0 1 1 1 

M
{0: 3, 1: 1, 2: 1}



In [3]:
X = {}
for p in range(P):
    for c in range(C):
        for s in range(S):
            for t in range(T):
                X[p,c,s,t] = horario.BoolVar(f'X[{p}][{c}][{s}][{t}]')

O lider participa em todas as reuniões do projeto:

$$
\forall_{p< P,c< C,s< S,t<T }  \quad x_{p,l_p,s,t} \ge x_{p,c,s,t}
$$

In [4]:
for p in range(P):
    for c in range(C):
        for s in range(S):
            for t in range(T):
                horario.Add(X[p,l[p],s,t] >= X[p,c,s,t])


O projeto tem mínimo (“quorum”) de 50% do total de colaboradores do projeto:
$$
\forall_{p< P,s< S,t< T} \quad \sum_{c< C} x_{p,c,s,t}\ge \frac{\sum_{c< C} col_{p,c}}{2}.x_{p,l_p,s,t}
$$

In [5]:
for p in range(P):
    for s in range(S):
        for t in range(T):
            horario.Add(sum([ X[p,c,s,t] for c in range(C)]) >= sum([ col[p,c] for c in range(C) ])*X[p,l[p],s,t]/2)
                

Verificar se o colaborador tem disponibilidade:
$$
\forall_{c< C,t< T} \quad d_{c,t} = 0 \quad \implies \quad \sum_{p<P,s< S} x_{p,c,s,t} = 0
$$

In [6]:
for c in range(C):
    for t in range(T):
        if d[c,t] == 0:
            horario.Add(sum([ X[p,c,s,t] for p in range(P) for s in range(S)]) == 0)

Número Mínimo de reuniões semanais:
$$
\forall_{p< P} \quad \sum_{s< S,t< T} x_{p,l_p,s,t} \ge m_p
$$

In [7]:
for p in range(P):
    horario.Add(sum([ X[p,l[p],s,t] for c in range(C) for s in range(S) for t in range(T)]) >= m[p])

Colaboradores não podem participar em reuniões de projetos que não estão associados:
$$
\forall_{p< P,c< C,s< S,t< T} \quad x_{p,c,s,t} \le col_{p,c}
$$

In [8]:
for p in range(P):
    for c in range(C):
        for s in range(S):
            for t in range(T):
                horario.Add(X[p,c,s,t] <= col[p,c])

Um colaborador não pode participar em duas reuniões ao mesmo tempo:
$$
\forall_{c< C, t< T} \quad\sum_{p< P, s< S} x_{p,c,s,t} \le 1
$$

In [9]:
for c in range(C):
    for t in range(T):
        horario.Add(sum([ X[p,c,s,t] for p in range(P) for s in range(S)]) <= 1)

In [10]:
# S = Int
# C = Int
# T = (hora, dia)
# P = ([conjunto de colaboradores], lider, número de reuniões semanais)

# Maximizar o número de reuniões efetivamente realizadas
# Minimizar o número médio de reuniões por participante.

In [20]:
horario.Maximize(sum([X[p,l[p],s,t] for p in range(P) for s in range(S) for t in range(T)]))

status = horario.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print(1)
if status == pywraplp.Solver.INFEASIBLE:
    print(2)
if status == pywraplp.Solver.UNBOUNDED:
    print(3)

option = 1

if option == 0:
    for c in range(C):
        print(f'colaborador {c}:')
        for p in range(P):
            for s in range(S):
                for t in range(T):
                    if (int(X[p,c,s,t].solution_value()) == 1):
                        print(f'reunião do projeto {p} na sala {s} no slot {t}')
        print()
        print()
    
if option == 1:
    for p in range(P):
        n = 1
        print(f'projeto {p}:')
        for c in range(C):
            for s in range(S):
                for t in range(T):
                    if (int(X[p,c,s,t].solution_value()) == 1):
                        print(f'o colaborador {c} tem reunião na sala {s} no slot {t}')
        print()
        print()

1
projeto 0:
o colaborador 0 tem reunião na sala 0 no slot 1
o colaborador 0 tem reunião na sala 1 no slot 3
o colaborador 0 tem reunião na sala 1 no slot 4
o colaborador 0 tem reunião na sala 2 no slot 0
o colaborador 1 tem reunião na sala 0 no slot 1
o colaborador 1 tem reunião na sala 1 no slot 3
o colaborador 1 tem reunião na sala 1 no slot 4
o colaborador 2 tem reunião na sala 2 no slot 0


projeto 1:
o colaborador 2 tem reunião na sala 0 no slot 4
o colaborador 2 tem reunião na sala 1 no slot 1
o colaborador 2 tem reunião na sala 1 no slot 5
o colaborador 3 tem reunião na sala 0 no slot 2
o colaborador 3 tem reunião na sala 0 no slot 4
o colaborador 3 tem reunião na sala 1 no slot 1
o colaborador 3 tem reunião na sala 1 no slot 5
o colaborador 4 tem reunião na sala 0 no slot 2
o colaborador 4 tem reunião na sala 1 no slot 1
o colaborador 4 tem reunião na sala 1 no slot 5
o colaborador 5 tem reunião na sala 0 no slot 2
o colaborador 6 tem reunião na sala 0 no slot 4


projeto 2:
o